<a href="https://colab.research.google.com/github/edwinibague/Proyecto_De_Grado/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import tensorflow as tf
from tensorflow.keras.losses import categorical_crossentropy


# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
NameBen = '/content/drive/My Drive/DataMB/DataBen.pkl'
NameMal = '/content/drive/My Drive/DataMB/DataMal.pkl'

In [ ]:
DataBen = pd.read_pickle(NameBen)
DataMal = pd.read_pickle(NameMal)


In [ ]:
#--------- Generando lista de listas para sequence.pad_sequences
from typing import List
LDB = []
Datx = DataBen.transpose()
for x in range(len(DataBen)):
    LDB.append((Datx[x]))

In [ ]:
LDM = []
Daty = DataMal.transpose()
for x in range(len(DataMal)):
  LDM.append((Daty[x]))

#--------- Fin de transformacion de listas

In [ ]:
"""
with open('Malignas.json') as file:
    Malignas = json.load(file)

with open('Benignas.json') as file:
    Benignas = json.load(file)"""

    # load the dataset but only keep the top n words, zero the rest
#top_words = 5000
#(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
#(X_train, y_train), (X_test, y_test) = imdb.load_data(DataBen.all(),DataMal.all())


"\nwith open('Malignas.json') as file:\n    Malignas = json.load(file)\n\nwith open('Benignas.json') as file:\n    Benignas = json.load(file)"

In [ ]:
# load the dataset but only keep the top n words, zero the rest
#top_words = 5000
#(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
#(X_train, y_train), (X_test, y_test) = imdb.load_data(DataBen.all(),DataMal.all())


#Ben = LDB.truncate(before=0, after=(len(LDM)-1))
LDBn = LDB[0:len(LDM)]
X_train, X_test, _, _ = train_test_split(LDBn,LDBn)
y_train, y_test, _, _ = train_test_split(LDM,LDM)
print(len(y_train))
print(len(X_train[0]))

1713
360


In [ ]:
y_train = np.array(y_train, dtype= np.uint8)
y_test = np.array(y_test, dtype= np.uint8)
X_train =  np.array(X_train, dtype= np.uint8)
X_test = np.array(X_test, dtype= np.uint8)

In [ ]:
#X_train

In [ ]:
#y_train = y_train.transpose()
#X_train = X_train.transpose()

In [ ]:
#Change to list of list list[List[]]
#X_train = X_train.transpose()
#X_train[0]

In [ ]:
# truncate and pad input sequences
top_words = len(LDBn)
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

y_train = sequence.pad_sequences(y_train, maxlen=max_review_length)
y_test = sequence.pad_sequences(y_test, maxlen=max_review_length)

X_train.shape

(1713, 500)

In [ ]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dense(32, activation='sigmoid'))
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           73088     
                                                                 
 lstm_1 (LSTM)               (None, 500, 50)           16600     
                                                                 
 dense_1 (Dense)             (None, 500, 32)           1632      
                                                                 
Total params: 91,320
Trainable params: 91,320
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
27/27 [==============================] - 3s 43ms/step - loss: nan - accuracy: 0.2876 - val_loss: nan - val_accuracy: 0.3005
Epoch 2/10
27/27 [==============================] - 1s 26ms/step - loss: nan - accuracy: 0.2986 - val_loss: nan - val_accuracy: 0.3005
Epoch 3/10
27/27 [=====================

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 30.05%


In [ ]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(32, activation='softmax'))
model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=10, batch_size=64)


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 500, 32)           73088     
                                                                 
 lstm_14 (LSTM)              (None, 500, 50)           16600     
                                                                 
 dropout_14 (Dropout)        (None, 500, 50)           0         
                                                                 
 lstm_15 (LSTM)              (None, 100)               60400     
                                                                 
 dropout_15 (Dropout)        (None, 100)               0         
                                                                 
 dense_9 (Dense)             (None, 32)                3232      
                                                                 
Total params: 153,320
Trainable params: 153,320
Non-tr

InvalidArgumentError: ignored

In [ ]:
# LSTM with Dropout for sequence classification in the IMDB dataset

# fix random seed for reproducibility
#numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
#LDBn = LDB[0:len(LDM)]
#X_train, X_test, _, _ = train_test_split(LDBn,LDBn)
#y_train, y_test, _, _ = train_test_split(LDM,LDM)
#y_train = np.array(y_train, dtype= np.uint8)
#y_test = np.array(y_test, dtype= np.uint8)
#X_train =  np.array(X_train, dtype= np.uint8)
#X_test = np.array(X_test, dtype= np.uint8)
# truncate and pad input sequences
top_words = len(LDBn)
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

y_train = sequence.pad_sequences(y_train, maxlen=max_review_length)
y_test = sequence.pad_sequences(y_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 32)           73088     
                                                                 
 dropout_2 (Dropout)         (None, 500, 32)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               53200     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 126,389
Trainable params: 126,389
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3


InvalidArgumentError: ignored

In [ ]:
# LSTM with dropout for sequence classification in the IMDB dataset

# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 471s 1s/step - loss: 0.5063 - accuracy: 0.7403
Epoch 2/3
391/391 [==============================] - 469s 1s/step - loss: 0.3240 - accuracy: 0.8671
Epoch 3/3
391/391 [==============================] - 467s 1s/step - loss: 0.2612 - accuracy: 0.8956
Accuracy: 87.25%


In [ ]:
# LSTM and CNN for sequence classification in the IMDB dataset

# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 250, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 132s 330ms/step - loss: 0.4226 - accuracy: 0.7901
Epoch 